In [1]:
#Dependencies
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



#data loading
data = load_breast_cancer()
X = data.data
y = data.target
#print(X)
#print(y[:5])




#train test split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
#print(X_train.shape, X_test.shape)
#print(y_train.shape, y_test.shape)




#standardising
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)
typex = type(X_train)
print("X_train_scaled is of", typex, "type")



X_train_scaled is of <class 'numpy.ndarray'> type


In [2]:
#Convert normal data to pytorch tensor data and point it to colab's gpu

X_train = torch.tensor(X_train, dtype = torch.float32)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.float32)
y_test = torch.tensor(y_test, dtype = torch.float32)


In [3]:
# NN Arch

class neural_network(nn.Module):
 
  def __init__(self, input_size, hidden_size, output_size):
    super(neural_network, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, x):
    out = self.fc1(x)
    out = self.relu (out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out
  

In [4]:
# parameters:

input_size = X_train.shape[1]
hidden_size = 64
output_size = 1 #only 1 op is expected
learning_rate = 0.001
number_epoch = 100

In [5]:
# initialise model

model = neural_network(input_size, hidden_size, output_size)

In [6]:
#define loss and optimiser

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [7]:
#Training NN

for epoch in range(number_epoch):
    model.train()
    optimizer.zero_grad() 
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1,1))
    loss.backward()
    optimizer.step()

    # accuracy
    with torch.no_grad():
        predicted = outputs.round()
        correct = (predicted == y_train.view(-1,1)).float().sum()
        accuracy = correct/y_train.size(0)
    
    if (epoch + 1) % 10 == 0: #how each epoch runs
        print(f"Epoch [{epoch+1}/{number_epoch}], Loss : {loss.item():.4f}, Train Accuracy : {accuracy.item()*100:.2f}%")


Epoch [10/100], Loss : 0.5422, Train Accuracy : 90.77%
Epoch [20/100], Loss : 0.4194, Train Accuracy : 91.87%
Epoch [30/100], Loss : 0.3280, Train Accuracy : 93.19%
Epoch [40/100], Loss : 0.2613, Train Accuracy : 94.73%
Epoch [50/100], Loss : 0.2142, Train Accuracy : 94.95%
Epoch [60/100], Loss : 0.1812, Train Accuracy : 95.38%
Epoch [70/100], Loss : 0.1576, Train Accuracy : 96.26%
Epoch [80/100], Loss : 0.1401, Train Accuracy : 97.36%
Epoch [90/100], Loss : 0.1266, Train Accuracy : 97.36%
Epoch [100/100], Loss : 0.1159, Train Accuracy : 97.36%


In [8]:
#evaluation:

model.eval()

with torch.no_grad():
    outputs = model(X_test)
    predicted = outputs.round()
    correct = (predicted == y_test.view(-1,1)).float().sum()
    accuracy = correct/y_test.size(0)
    print(f"Accuracy on testing data: {accuracy.item() * 100:.2f}%")

Accuracy on testing data: 97.37%


In [9]:
torch.save(model.state_dict(), "brca_NN_pytorch.pth")


In [ ]:
# loading

model2 = neural_network(input_size, hidden_size, output_size)
model2.load_state_dict(torch.load("brca_NN_pytorch.pth"))
model2.eval()